In [1]:
import sys
sys.path.append("/Users/chenjieli/Desktop/RuleCleaner/") 

In [2]:
import pickle
import glob
import re
from rulecleaner_src.lfs_tree import keyword_labelling_func_builder
from rulecleaner_src.TreeRules import SPAM, HAM, ABSTAIN, PredicateNode
from rulecleaner_src.LFRepair import populate_violations, fix_rules_with_solver_input
from rulecleaner_src.utils import run_snorkel_with_funcs, select_user_input, clean_text
import psycopg2
import pandas as pd
from snorkel.labeling import (
	LabelingFunction, 
	labeling_function, 
	PandasLFApplier, 
	LFAnalysis,
	filter_unlabeled_dataframe
	)
from snorkel.labeling.model import MajorityLabelVoter, LabelModel
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.patches as mpatches
import pulp
from sklearn.metrics import accuracy_score, classification_report
import random
from collections import deque, defaultdict
import numpy as np
from IPython.display import Image, display 
import datetime
import itertools

from rulecleaner_src.main import main

[nltk_data] Downloading package words to /Users/chenjieli/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chenjieli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from rulecleaner_src.lfs_tree import keyword_labelling_func_builder, regex_func_builder

In [4]:
def filter_invalid_regex(regex_str):
    # print(f'testing : {regex_str}')
    try:
        re.compile(regex_str)
        # print("Regex is valid!")
        return regex_str
    except re.error as e:
        print(f"Regex error: {e} for pattern {regex_str}")
        return None

In [5]:
raw_responses = {}

In [6]:
for fname in glob.glob('lf_raw_response_datase*_new.pkl'):
    print(fname)
    match = re.match(r'lf_raw_response_datase_(\w+)_new', fname)
    dataset_name = match.group(1)
    with open(fname, 'rb') as f:
        raw_responses[dataset_name] = pickle.load(f)

lf_raw_response_datase_agnews_new.pkl
lf_raw_response_datase_plots_new.pkl
lf_raw_response_datase_amazon_new.pkl
lf_raw_response_datase_imdb_new.pkl
lf_raw_response_datase_tweets_new.pkl
lf_raw_response_datase_spam_new.pkl
lf_raw_response_datase_yelp_new.pkl
lf_raw_response_datase_painter_architect_new.pkl
lf_raw_response_datase_fakenews_new.pkl
lf_raw_response_datase_professor_teacher_new.pkl
lf_raw_response_datase_physician_professor_new.pkl


In [7]:
with open('lf_raw_response_datase_spam_new.pkl', 'rb') as f:
    spam_raw = pickle.load(f)

In [8]:
for sr in spam_raw:
    print(sr)

Based on the provided sentences and their labels, we can create labeling functions using both keyword and regular expression templates. Here are some potential labeling functions:

### Keyword-based Labeling Functions

```python
def keyword_spam1(x):
    ABSTAIN = -1
    keywords = ["free", "winner", "claim", "reward", "guaranteed", "prize", "congratulations", "selected", "entry", "win"]
    return 1 if any(keyword.lower() in x.lower() for keyword in keywords) else ABSTAIN

def keyword_spam2(x):
    ABSTAIN = -1
    keywords = ["call", "text", "reply", "send", "msg", "message", "sms", "chat", "stop", "unsubscribe"]
    return 1 if any(keyword.lower() in x.lower() for keyword in keywords) else ABSTAIN

def keyword_spam3(x):
    ABSTAIN = -1
    keywords = ["£", "pound", "cash", "money", "award", "voucher", "points", "balance", "account", "bonus"]
    return 1 if any(keyword.lower() in x.lower() for keyword in keywords) else ABSTAIN

def keyword_spam4(x):
    ABSTAIN = -1
    keywords = 

In [9]:
used_datasets = [
    'plots',
    'spam',
    'tweets',
    'painter_architect',
    'professor_teacher',
    'yelp',
    'fakenews',
    'imdb',
    'physician_professor',
    'agnews',
    'amazon'
]




conn = psycopg2.connect(
    dbname="label",
    user="postgres",
)

mapping_dicts = {}

for ud in used_datasets:
    print(ud)
    info_df = pd.read_sql(f'select count(*), class, label from {ud} group by class, label', conn)
    print(info_df)
    result = dict(zip(info_df['label'], info_df['class']))
    mapping_dicts[ud]= result
    print('\n')


plots
   count  class    label
0    915      0   action
1   1030      1  romance


spam
   count  class label
0    747      1  spam
1   4825      0   ham


tweets
   count  class     label
0   2363      0  positive
1   9178      1  negative


painter_architect
   count  class      label
0   6118      0    painter
1   6118      1  architect


professor_teacher
   count  class      label
0  12294      1  professor
1  12294      0    teacher


yelp
   count  class     label
0  19000      0  negative
1  19000      1  positive


fakenews
   count  class label
0  21417      0  true
1  23481      1  fake


imdb
   count  class     label
0  25000      0  negative
1  25000      1  positive


physician_professor
   count  class      label
0  27238      1  physician
1  27238      0  professor


agnews
   count  class       label
0  30000      0  Technology
1  30000      1    Business


amazon


/var/folders/sl/1_4_3jg90p50ft92b5k6mgv00000gn/T/ipykernel_4553/3965635007.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  info_df = pd.read_sql(f'select count(*), class, label from {ud} group by class, label', conn)
/var/folders/sl/1_4_3jg90p50ft92b5k6mgv00000gn/T/ipykernel_4553/3965635007.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  info_df = pd.read_sql(f'select count(*), class, label from {ud} group by class, label', conn)
/var/folders/sl/1_4_3jg90p50ft92b5k6mgv00000gn/T/ipykernel_4553/3965635007.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not t

    count  class     label
0   99936      0  positive
1  100064      1  negative




In [10]:
mapping_dicts

{'plots': {'action': 0, 'romance': 1},
 'spam': {'spam': 1, 'ham': 0},
 'tweets': {'positive': 0, 'negative': 1},
 'painter_architect': {'painter': 0, 'architect': 1},
 'professor_teacher': {'professor': 1, 'teacher': 0},
 'yelp': {'negative': 0, 'positive': 1},
 'fakenews': {'true': 0, 'fake': 1},
 'imdb': {'negative': 0, 'positive': 1},
 'physician_professor': {'physician': 1, 'professor': 0},
 'agnews': {'Technology': 0, 'Business': 1},
 'amazon': {'positive': 0, 'negative': 1}}

In [11]:
allowed_labels = {}

In [12]:
for k,v in mapping_dicts.items():
    allowed_labels[k] = set(v.values())

In [13]:
allowed_labels

{'plots': {0, 1},
 'spam': {0, 1},
 'tweets': {0, 1},
 'painter_architect': {0, 1},
 'professor_teacher': {0, 1},
 'yelp': {0, 1},
 'fakenews': {0, 1},
 'imdb': {0, 1},
 'physician_professor': {0, 1},
 'agnews': {0, 1},
 'amazon': {0, 1}}

In [14]:

import re

def parse_labeling_functions(text: str):
    """
    Parses multiple keyword-based and regex-based labeling functions from a text file.
    Returns a list of extracted function details.
    """
    functions = []

    # Match all function definitions (both keyword & regex)
    function_matches = re.findall(r'\bdef (keyword|regex)_(\w+)(\d+)\(x\):', text, re.IGNORECASE)
    
    # print(f"DEBUG: Found {len(function_matches)} functions.")  # Print function count
    
    for lf_type, category, label in function_matches:
        expected_label = int(label)

        # Extract the full function block (handling multi-line functions)
        function_block_match = re.search(
            rf'def {lf_type}_{category}{label}\(x\):([\s\S]+?)(?=\n\s*def |\Z)', text, re.IGNORECASE
        )

        if not function_block_match:
            print(f"ERROR: Could not extract function body for {lf_type}_{category}{label}")
            continue

        function_body = function_block_match.group(1)
        # print(f"DEBUG: Extracted function {lf_type}_{category}{label}")

        # Extract keywords for keyword-based functions
        if lf_type == "keyword":
            keywords_match = re.search(r'keywords\s*=\s*\[(.*?)\]', function_body, re.DOTALL)
            if keywords_match:
                keywords = [kw.strip(" '\"") for kw in keywords_match.group(1).split(',')]
                functions.append({
                    "type": "keyword",
                    "category": category.lower(),
                    "keywords": keywords,
                    "expected_label": expected_label
                })
            else:
                print(f"ERROR: Could not extract keywords for {lf_type}_{category}{label}")

        # Extract regex patterns for regex-based functions
        elif lf_type == "regex":
            regex_patterns = []

            # Extract `re.search(r'pattern', x, re.IGNORECASE)` occurrences
            regex_match = re.findall(r"re\.search\(\s*r?['\"](.+?)['\"],\s*x", function_body, re.DOTALL)
            regex_patterns.extend(regex_match)

            # Extract single regex pattern assigned to a variable
            single_pattern_match = re.findall(r"pattern\s*=\s*r?['\"](.+?)['\"]", function_body)
            regex_patterns.extend(single_pattern_match)

            # Extract multi-line regex list (`patterns = [...]`)
            pattern_list_match = re.search(r"\s*patterns\s*=\s*\[(.*?)\]", function_body, re.DOTALL)
            if pattern_list_match:
                patterns_raw = pattern_list_match.group(1)

                # Extract individual patterns inside the list
                patterns_cleaned = re.findall(r"r?['\"](.+?)['\"]", patterns_raw, re.DOTALL)
                patterns_cleaned = [p.replace("\\'", "'") for p in patterns_cleaned]  # Fix escaped single quotes
                regex_patterns.extend(patterns_cleaned)

            # Debugging output
            # print(f"DEBUG: Extracted patterns for {lf_type}_{category}{label}: {regex_patterns}")

            # Ensure all valid regex patterns are captured
            if regex_patterns:
                functions.append({
                    "type": "regex",
                    "category": category.lower(),
                    "patterns": regex_patterns,
                    "expected_label": expected_label
                })
            else:
                print(f"ERROR: Could not extract regex patterns for {lf_type}_{category}{label}")

    # print(f"DEBUG: Successfully parsed {len(functions)} functions.")
    return functions, len(function_matches)

In [15]:
info_res = []

In [16]:
lf_construct_infos = {}

In [17]:
for k, v in raw_responses.items():
    if(k in used_datasets):
        lf_construct_infos[k] = []
        for r in raw_responses[k]:
            infos, num_matches = parse_labeling_functions(r)
            # for f in infos:
            #     if(f['type']=='regex' and len(f['patterns'])>1):
            #         print(r)
            #         print(infos)
            if(len(infos)!=num_matches):
                print(f"didnt parse all functions succesfully, the text has {num_matches}, but it only parsed {len(infos)}")
                # print(f"original text")
                print(r)
                # print("infos")
                print("returned result")
                print(infos)
                print('*******'*20)
                continue
            # else:
            #     print(f"succes, the text has {num_matches},and parsed {len(infos)}")
            #     # print(f"original text")
            #     print(r)
            #     # print("infos")
            #     print("returned result")
            #     print(infos)
            #     print('*******'*20)
            #     # continue         
    
            lf_construct_infos[k].extend(infos)

ERROR: Could not extract regex patterns for regex_positive_1
didnt parse all functions succesfully, the text has 4, but it only parsed 3
Based on the provided sentences and their labels, here are some labeling functions using the specified templates:

### Keyword-based Labeling Functions

```python
def keyword_negative_0(x):
    ABSTAIN = -1
    keywords = ["disappointing", "awful", "boring", "predictable", "lame", "misfortune", "terrible", "dreadful", "uninteresting", "cheesy"]
    return 0 if any(keyword in x.lower() for keyword in keywords) else ABSTAIN

def keyword_positive_1(x):
    ABSTAIN = -1
    keywords = ["hilarious", "enjoyable", "great", "funny", "entertaining", "well-crafted", "extraordinary", "amazing", "refreshing", "magical"]
    return 1 if any(keyword in x.lower() for keyword in keywords) else ABSTAIN
```

### Regular Expression-based Labeling Functions

```python
def regex_negative_0(x):
    ABSTAIN = -1
    import re
    patterns = [
        r"worst film.*ever",  #

In [18]:
df_lf_metadata = pd.read_csv('dataset_lf_meta_data.csv')
df_lf_metadata = df_lf_metadata.iloc[:, 1:]  # Drop the first column

In [19]:
lf_metadata_dict = df_lf_metadata.to_dict(orient='records')

In [20]:
dataset_dict = {item['Dataset']: {k: v for k, v in item.items() if k != 'Dataset'} for item in lf_metadata_dict}


In [21]:
for k,v in lf_construct_infos.items():
    print(f"{k}: {len(v)}")

agnews: 78
plots: 55
amazon: 80
imdb: 76
tweets: 72
spam: 52
yelp: 80
painter_architect: 73
fakenews: 96
professor_teacher: 72
physician_professor: 81


In [22]:
for k,v in lf_construct_infos.items():
    lf_construct_infos[k] = [x for x in lf_construct_infos[k] if x['expected_label'] in allowed_labels[k]]

In [23]:
import random

def sample_from_list(data_list, sample_size, seed=42):
    random.seed(seed)  # Set seed for reproducibility
    return random.sample(data_list, sample_size)

In [24]:
dataset_dict

{'amazon': {'Size': 200000,
  'AvgTokenPerText': 68.9,
  'Classes': 'positive/negative',
  'LFCount': 23},
 'agnews': {'Size': 60000,
  'AvgTokenPerText': 37.7,
  'Classes': 'business/technology',
  'LFCount': 21},
 'physician_professor': {'Size': 54476,
  'AvgTokenPerText': 55.8,
  'Classes': 'physician/professor',
  'LFCount': 20},
 'imdb': {'Size': 50000,
  'AvgTokenPerText': 230.7,
  'Classes': 'positive/negative',
  'LFCount': 20},
 'fakenews': {'Size': 44898,
  'AvgTokenPerText': 405.9,
  'Classes': 'true/false',
  'LFCount': 20},
 'yelp': {'Size': 38000,
  'AvgTokenPerText': 133.6,
  'Classes': 'negative/positive',
  'LFCount': 20},
 'professor_teacher': {'Size': 24588,
  'AvgTokenPerText': 62.2,
  'Classes': 'professor/teacher',
  'LFCount': 19},
 'painter_architect': {'Size': 12236,
  'AvgTokenPerText': 62.6,
  'Classes': 'painter/architect',
  'LFCount': 18},
 'tweets': {'Size': 11541,
  'AvgTokenPerText': 18.5,
  'Classes': 'positive/negative',
  'LFCount': 18},
 'spam': {'S

In [25]:
for k,v in lf_construct_infos.items():
    lf_construct_infos[k] = sample_from_list(lf_construct_infos[k], dataset_dict[k]['LFCount'], seed=123)

In [26]:
for k,v in lf_construct_infos.items():
    print(f"{k}: {len(v)}")

agnews: 21
plots: 14
amazon: 23
imdb: 20
tweets: 18
spam: 16
yelp: 20
painter_architect: 18
fakenews: 20
professor_teacher: 19
physician_professor: 20


In [25]:
lf_construct_infos

{'tweets': [{'type': 'regex',
   'category': 'positive_',
   'patterns': ['\\b(thank(s| you)|love|appreciate|excellent|stellar|rocks|sweethearts|amazing|applause)\\b'],
   'expected_label': 0},
  {'type': 'regex',
   'category': 'positive',
   'patterns': ['\\b(thank(s| you)|pleasure|amazing|great(est)?|excellent|outstanding|smile|quick reply)\\b'],
   'expected_label': 0},
  {'type': 'regex',
   'category': 'negative_',
   'patterns': ['\\b(waiting|complaint|disappointed|sucks|shitty|crazy|negligence|not good|stole|inconceivable)\\b'],
   'expected_label': 1},
  {'type': 'keyword',
   'category': 'negative',
   'keywords': ['Cancelled Flight',
    'delayed',
    'waiting',
    'on hold',
    'confused',
    'no baggage',
    'unbelievable',
    "can't",
    'cut off',
    'further delayed'],
   'expected_label': 1},
  {'type': 'regex',
   'category': 'positive',
   'patterns': ['\\b(thank(s| you)|great|fantastic|keep up the good work|prompt response)\\b'],
   'expected_label': 0},
  {

In [26]:

user_input_sizes = [10]
random_states = [4]
lf_acc_threshs = [1]
instance_acc_threshs = [1]
non_abstain_threshs = [1]
datasets = ['spam']
func_dictionary = [lf_construct_infos]
instance_acc_on_valids=[False]
use_non_abstains=[False]
pfile_name_prefix = ('test_folder/test_run',)
lf_source = ['gpt_generated']


In [27]:
input_params = list(itertools.product(
    user_input_sizes,
    lf_acc_threshs,
    instance_acc_threshs,
    non_abstain_threshs,
    datasets,
    random_states,
    func_dictionary,
    instance_acc_on_valids,
    use_non_abstains,
    pfile_name_prefix,
    lf_source
))

In [28]:
for k,v in func_dictionary[0].items():
    print(f"{k}:{len(v)}")

tweets:18
spam:16
yelp:20
painter_architect:18
fakenews:20
professor_teacher:19
physician_professor:20


In [29]:
for k in func_dictionary[0]:
    print(f'filtering rules for {k}')
    for i in range(len(func_dictionary[0][k])):
        if(func_dictionary[0][k][i]['type']=='regex'):
            for j in range(len(func_dictionary[0][k][i]['patterns'])):
                func_dictionary[0][k][i]['patterns'][j] = filter_invalid_regex(func_dictionary[0][k][i]['patterns'][j])
            func_dictionary[0][k][i]['patterns'] = [x for x in func_dictionary[0][k][i]['patterns'] if x is not None]
                

filtering rules for tweets
Regex error: missing ), unterminated subpattern at position 2 for pattern \b(?:invalid.*itineraries|can
filtering rules for spam
filtering rules for yelp
filtering rules for painter_architect
filtering rules for fakenews
filtering rules for professor_teacher
filtering rules for physician_professor


In [30]:
for k in func_dictionary[0]:
    print(f'filtering rules for {k}')
    for i in range(len(func_dictionary[0][k])):
        if(func_dictionary[0][k][i]['type']=='regex'):
            for j in range(len(func_dictionary[0][k][i]['patterns'])):
                func_dictionary[0][k][i]['patterns'][j] = filter_invalid_regex(func_dictionary[0][k][i]['patterns'][j])
            func_dictionary[0][k][i]['patterns'] = [x for x in func_dictionary[0][k][i]['patterns'] if x is not None]
                

filtering rules for tweets
filtering rules for spam
filtering rules for yelp
filtering rules for painter_architect
filtering rules for fakenews
filtering rules for professor_teacher
filtering rules for physician_professor


In [31]:
for k,v in func_dictionary[0].items():
    print(f"{k}:{len(v)}")

tweets:18
spam:16
yelp:20
painter_architect:18
fakenews:20
professor_teacher:19
physician_professor:20


In [35]:
func_dictionary[0]['spam']

[{'type': 'keyword',
  'category': 'ham',
  'keywords': ['sorry',
   'please',
   'home',
   'call',
   'message',
   'buy',
   'talk',
   'problem',
   'help',
   'ask'],
  'expected_label': 0},
 {'type': 'regex',
  'category': 'ham',
  'patterns': ['\\b\\d{3}\\b'],
  'expected_label': 0},
 {'type': 'regex',
  'category': 'ham',
  'patterns': ['\\b\\d+ bucks\\b',
   '\\b\\d+ dollars\\b',
   '\\b\\d+ lacs\\b',
   '\\b\\d+ yrs\\b',
   '\\b\\d+ problems\\b',
   '\\b\\d+ weeks\\b',
   '\\b\\d+ days\\b',
   '\\b\\d+ months\\b',
   '\\b\\d+ years\\b',
   '\\b\\d+ times\\b'],
  'expected_label': 0},
 {'type': 'keyword',
  'category': 'ham',
  'keywords': ['please',
   'love',
   'friends',
   'sorry',
   'special',
   'nice',
   'song',
   'time',
   'meet',
   'watch'],
  'expected_label': 0},
 {'type': 'keyword',
  'category': 'ham',
  'keywords': ['pick up',
   'possession',
   'fight',
   'princess',
   'question',
   'mad',
   'sleep',
   'free',
   'bottle',
   'respond'],
  'expected_

In [32]:
for ip in input_params:
    main(*ip)

user_input_size = 10
lf_acc_thresh = 1
instance_acc_thresh = 1
min_non_abstain_thresh = 1
dataset_name = spam
random_state = 4
funcs_dictionary = {'tweets': [{'type': 'regex', 'category': 'positive_', 'patterns': ['\\b(thank(s| you)|love|appreciate|excellent|stellar|rocks|sweethearts|amazing|applause)\\b'], 'expected_label': 0}, {'type': 'regex', 'category': 'positive', 'patterns': ['\\b(thank(s| you)|pleasure|amazing|great(est)?|excellent|outstanding|smile|quick reply)\\b'], 'expected_label': 0}, {'type': 'regex', 'category': 'negative_', 'patterns': ['\\b(waiting|complaint|disappointed|sucks|shitty|crazy|negligence|not good|stole|inconceivable)\\b'], 'expected_label': 1}, {'type': 'keyword', 'category': 'negative', 'keywords': ['Cancelled Flight', 'delayed', 'waiting', 'on hold', 'confused', 'no baggage', 'unbelievable', "can't", 'cut off', 'further delayed'], 'expected_label': 1}, {'type': 'regex', 'category': 'positive', 'patterns': ['\\b(thank(s| you)|great|fantastic|keep up the g

/Users/chenjieli/Desktop/RuleCleaner/rulecleaner_src/utils.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sentences_df=pd.read_sql(f'SELECT * FROM {dataset_name}', conn)
2025-02-01 12:26:41,979 - root - INFO - Computing O...
2025-02-01 12:26:41,981 - root - INFO - Estimating \mu...


initial_vectors: (5499, 16)
initial_vectors:
 [[-1 -1 -1 ...  1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 [ 0  0 -1 ...  1 -1  0]
 ...
 [-1 -1 -1 ... -1 -1 -1]
 [ 0 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]]


2025-02-01 12:26:42,428 - root - INFO - [0 epochs]: TRAIN:[loss=1.482]              | 0/500 [00:00<?, ?epoch/s]
2025-02-01 12:26:42,456 - root - INFO - [100 epochs]: TRAIN:[loss=0.037]
2025-02-01 12:26:42,483 - root - INFO - [200 epochs]: TRAIN:[loss=0.011]
2025-02-01 12:26:42,510 - root - INFO - [300 epochs]: TRAIN:[loss=0.010]
2025-02-01 12:26:42,538 - root - INFO - [400 epochs]: TRAIN:[loss=0.010]| 327/500 [00:00<00:00, 3265.85epoch/s]
100%|███████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 3393.55epoch/s]
2025-02-01 12:26:42,566 - root - INFO - Finished Training
2025-02-01 12:26:42,628 - pulp.apis.core - DEBUG - /Users/chenjieli/.pyenv/versions/3.10.0/envs/rulecleaner/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/sl/1_4_3jg90p50ft92b5k6mgv00000gn/T/42a80bd1bea1412f915d329c1578d0e2-pulp.mps -sec 600 -timeMode elapsed -branch -printingOptions all -solution /var/folders/sl/1_4_3jg90p50ft92b5k6mgv00000gn/T/42a80bd1bea141


        out of 5499 sentences, 5498 actually got at least one signal to 

        make prediction. Out of all the valid predictions, we have 3845 wrong predictions, 

        accuracy = 0.30065478355765735 
    
lf_acc: 1, ins_acc:1, min_non_abstain_thresh:1
labeling_functions: ['lf_1', 'lf_2', 'lf_3', 'lf_4', 'lf_5', 'lf_6', 'lf_7', 'lf_8', 'lf_9', 'lf_10', 'lf_11', 'lf_12', 'lf_13', 'lf_14', 'lf_15', 'lf_16']
num_instances: 10
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/chenjieli/.pyenv/versions/3.10.0/envs/rulecleaner/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/sl/1_4_3jg90p50ft92b5k6mgv00000gn/T/42a80bd1bea1412f915d329c1578d0e2-pulp.mps -sec 600 -timeMode elapsed -branch -printingOptions all -solution /var/folders/sl/1_4_3jg90p50ft92b5k6mgv00000gn/T/42a80bd1bea1412f915d329c1578d0e2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1311 COLUMNS
At line 8992 RHS
At line 

2025-02-01 12:26:42,789 - rulecleaner_src.LFRepair - DEBUG - the fix: valid: gini score: 0.17777777777777778
2025-02-01 12:26:42,789 - rulecleaner_src.LFRepair - DEBUG - the fix: u
2025-02-01 12:26:42,789 - rulecleaner_src.LFRepair - DEBUG - the fix: know
2025-02-01 12:26:42,789 - rulecleaner_src.LFRepair - DEBUG - the fix: da
2025-02-01 12:26:42,790 - rulecleaner_src.LFRepair - DEBUG - the fix: feeling
2025-02-01 12:26:42,790 - rulecleaner_src.LFRepair - DEBUG - the fix: someone
2025-02-01 12:26:42,790 - rulecleaner_src.LFRepair - DEBUG - the fix: faster
2025-02-01 12:26:42,799 - rulecleaner_src.LFRepair - DEBUG - the fix: go
2025-02-01 12:26:42,805 - rulecleaner_src.LFRepair - DEBUG - the fix: find
2025-02-01 12:26:42,812 - rulecleaner_src.LFRepair - DEBUG - the fix: one
2025-02-01 12:26:42,821 - rulecleaner_src.LFRepair - DEBUG - the fix: group
2025-02-01 12:26:42,824 - rulecleaner_src.LFRepair - DEBUG - the fix: attached
2025-02-01 12:26:42,830 - rulecleaner_src.LFRepair - DEBUG - 

initial_vectors: (5499, 16)
initial_vectors:
 [[0 0 0 ... 1 0 0]
 [0 1 1 ... 1 0 0]
 [1 0 1 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


2025-02-01 12:26:46,982 - root - INFO - [0 epochs]: TRAIN:[loss=96.218]             | 0/500 [00:00<?, ?epoch/s]
2025-02-01 12:26:47,022 - root - INFO - [100 epochs]: TRAIN:[loss=0.170]
2025-02-01 12:26:47,049 - root - INFO - [200 epochs]: TRAIN:[loss=0.156]
2025-02-01 12:26:47,076 - root - INFO - [300 epochs]: TRAIN:[loss=0.155]
2025-02-01 12:26:47,105 - root - INFO - [400 epochs]: TRAIN:[loss=0.155]| 310/500 [00:00<00:00, 3099.85epoch/s]
100%|███████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 3277.55epoch/s]
2025-02-01 12:26:47,133 - root - INFO - Finished Training



        out of 5499 sentences, 5499 actually got at least one signal to 

        make prediction. Out of all the valid predictions, we have 834 wrong predictions, 

        accuracy = 0.8483360611020185 
    


In [33]:
# user_input_sizes = [80, 120, 150]
# random_states = [1,2]
# lf_acc_threshs = [0.7]
# instance_acc_threshs = [0.8]
# non_abstain_threshs = [0.8]
# datasets = list(lf_construct_infos)
# func_dictionary = [lf_construct_infos]
# instance_acc_on_valids=[False]
# use_non_abstains=[False]
# pfile_name_prefix = ('test_folder/test_run',)
# lf_source = 'gpt_generated'

# input_params2 = list(itertools.product(
#     user_input_sizes,
#     lf_acc_threshs,
#     instance_acc_threshs,
#     non_abstain_threshs,
#     datasets,
#     random_states,
#     func_dictionary,
#     instance_acc_on_valids,
#     use_non_abstains,
#     pfile_name_prefix,
#     lf_source
# ))

In [34]:
# for ip in input_params2:
#     main(*ip)